# Deploying the churn prediction project

# 5. Deploying machine learning models

Model deployment is the process of putting models to use. This chapter is about packaging a model inside a web service so that other services can use it. We will also see how to deploy the web service to a production-ready environment.

## 5.1. Churn-prediction model

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv("../chapter3_machine_learning_for_classification/Telco-Customer-Churn.csv")

df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df["TotalCharges"] = df["TotalCharges"].fillna(0)

df.columns = df.columns.str.lower().str.replace(" ", "_")
string_columns = list(df.dtypes[df.dtypes == "object"].index)
for column in string_columns:
    df[column] = df[column].str.lower().str.replace(" ", "_")

df.churn = (df.churn == "yes").astype(int)

In [3]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

df_train_full = df_train_full.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

y_train = df_train.churn.values
y_val = df_val.churn.values

del df_train["churn"]
del df_val["churn"]

In [4]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [5]:
def train(df, y, C=1.0):
    cat = df[categorical + numerical].to_dict(orient="records")
    
    dv = DictVectorizer(sparse=False)
    dv.fit(cat)

    X = dv.transform(cat)

    model = LogisticRegression(solver="liblinear", C=C)
    model.fit(X, y)

    return dv, model

def predict(df, dv, model):
    # applies the same one-hot encoding scheme as in the train function
    cat = df[categorical + numerical].to_dict(orient="records")
    
    X = dv.transform(cat)

    # uses the model to make predictions
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [6]:
y_train = df_train_full["churn"].values
y_test = df_test["churn"].values

dv, model = train(df_train_full, y_train, C=0.5)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)
print("auc = %.3f" % auc)

auc = 0.858


### 5.1.1. Using the model

In [7]:
customer = {
    'customerid': '8879-zkjof',
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'no',
    'dependents': 'no',
    'tenure': 41,
    'phoneservice': 'yes',
    'multiplelines': 'no',
    'internetservice': 'dsl',
    'onlinesecurity': 'yes',
    'onlinebackup': 'no',
    'deviceprotection': 'yes',
    'techsupport': 'yes',
    'streamingtv': 'yes',
    'streamingmovies': 'yes',
    'contract': 'one_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'bank_transfer_(automatic)',
    'monthlycharges': 79.85,
    'totalcharges': 3320.75
}

To predict whether this customer is going to churn, we can use the predict function.

In [8]:
df = pd.DataFrame([customer])
y_pred = predict(df, dv, model)
y_pred[0]

np.float64(0.05960531889903496)

The current predict function is inefficient if we're only using one customer with it because we turn that customer dictionary into a dataframe, only to convert that dataframe back to a dictionary later. We can create a separate function for predicting the probability of churn for a single customer.

In [9]:
def predict_single(customer, dv, model):
    X = dv.transform([customer])
    y_pred = model.predict_proba(X)[:, 1]
    return y_pred[0]

In [10]:
predict_single(customer, dv, model)

np.float64(0.05960531889903496)

The result is the same, this customer has a 6% probability of churning.

### 5.1.2. Using Pickle to save and load the model

To be able to use our models outside of a notebook, we need to save it, and then later, another process can load and use it.

Pickle is a serialization/deserialization module that's already built into Python. Using it, we can save an arbitrary Python object (with a few exceptions) to a file. Once we have a file, we can load the model from there in a different process.

Pickling an object in Python means saving it using the Pickle module.

To save a model, we first import the Pickle module, and then use the dump function:

In [11]:
import pickle

with open("churn-model.bin", "wb") as f_out:
    pickle.dump(model, f_out)

To save the model, we use the open function, which takes two arguments:

- The name of the file that we want to open. For us, it’s churn-model.bin.
- The mode with which we open the file. For us, it’s wb, which means we want to write to the file (w), and the file is binary (b) and not text—Pickle uses binary format for writing to files.

The open function returns f_out—the file descriptor we can use to write to the file.

Next, we use the dump function from Pickle. It also takes two arguments:

- The object we want to save. For us, it’s model.
- The file descriptor, pointing to the output file, which is f_out for us.

Finally, we use the with construction in this code. When we open a file with open, we need to close it after we finish writing. When using with, it happens automatically.

In our case, howevver, saving just the model is not enough: we also have a DictVectorizer that we "trained" together with the model. We need to save both.

The simplest way of doing this is to put borh of them in a tuple when pcikling:

In [12]:
with open("churn-model.bin", "wb") as f_out:
    pickle.dump((dv, model), f_out)

To load the model, we use the laod function from Pickle. We can test it in the same Jupyter Notebook:

In [13]:
with open("churn-model.bin", "rb") as f_in:
    dv, model = pickle.load(f_in)

We again use the open function, but this time, with a different mode: rb, which means we open it for reading (r), and the file is binary (b).

Be careful when specifying the mode. Accidentally specifying an incorrect mode may result in data loss: if you open an existing file with the w mode instead of r, it will overwrite the content.

Because we saved a tuple, we unpack it when loading, so we get both the vectorizer and the model at the same time.

Let's create a simple Python script that loads the model and applies it to a customer.

We will call this file churn_serving.py. It contains:

- The predict_simple function that we wrote earlier.
- The code for loading the model.
- The code for applying the model to a customer.

We can run the file like this:

In [14]:
%run churn_serving.py

prediction: 0.060
verdict: Not churn


This way, we can load the model and apply it to the customer we specified in the script.

## 5.2. Model serving

We already know how to load a trained model in a different process. Now we need to serve this model-make it available for others to use.

Here, we will do it in Python with Flask-a Python framework for creating web services.

### 5.2.1. Web services

We already know how to use a model to make a prediction, but so far, we have simply hardcoded the features of a customer as a Python dictionary. Let’s try to imagine how our model will be used in practice.

Suppose we have a service for running marketing campaigns. For each customer, it needs to determine the probability of churn, and if it's high enough, it will send a promotional email with discounts. Of course, this service needs to use our model to decide whether it should send an email.

One possible way of achieving this is to modify the code of the campaign service: load the model, and score the customers right in the service. This approach is good, but the campaign service needs to be in Python, and we need to have full control over its code.

Unfortunately, this situation is not always the case: it may be written in some other language, or a different team might be in charge of this project, which means we won’t have the control we need.

The typical solution for this problem is putting a model inside a web service—a small service (a microservice) that only takes care of scoring customers.

So, we need to create a churn service-a service in Python that will serve the churn model. Given the features of a customer, it will respond with the probability of churn for this customer. For each customer, the campaign service will ask the churn service for the probability of churn, and if it's high enough, then we send a promotional email.

This gives us another advantage: separation of concerns. If the model is created by data scientists, then they can take ownership of the service and maintain it, while the other team takes care of the campaign service.

One of the most popular frameworks for creating web services in Python is Flask.

### 5.2.2. Flask

The easiest way to implement a web service in Python is to use Flask. It's quite lightweight, requires little code to get started, and hides most of the complexity of dealing with HTTP requests and responses.

Let's make a Flask file and call it flask_test.py.

### 5.2.3. Serving churn model with Flask

In [32]:
import requests
url = 'http://localhost:9696/predict'
response = requests.post(url, json=customer)
result = response.json()
result

{'churn': False, 'churn_probability': 0.05960531889903496}

If the campaign service used Python, this is exactly how it could communicate with the churn service and decide who should get promotional emails.

## 5.3. Managing dependencies

### 5.3.1. Pipenv

To serve the churn model, we only need a few libraries: NumPy, Scikit-learn, and Flask.

Pipenv is a tool that makes managing virtual environments easier. We can install it with pip:

In [38]:
# pip install pipenv

After that, we use pipenv instead of pip for installing dependencies:

In [39]:
# pipenv install numpy scikit-learn flask

After all the libraries are installed, we need to activate the virtual environment-this way, our applicatoni will use the correct versions of the libraries. We do it by running the shell command:

In [40]:
# pipenv shell

Instead of first explicitly entering the virtual environment and then running the script, we can perform these two steps with just one command:

In [47]:
# pipenv run python churn_serving.py

The run command in Pipenv simply runs the specified program in the virtual environment.

### 5.3.2. Docker

Docker solves the "but it works on my machine" problem by aso packaging the OS and the system libraries into a Docker container–a self-contained environment that works anywhere where Docker is installed.

Once the service is packaged into a Docker container, we can run it on the host machine—our laptop (regardless of the OS) or any public cloud provider.

I won't do much Docker stuff because I want to focus on machine learning, but this chapter has some good stuff if you want to read it in the future.

Docker makes it easy to run services in a reproducible way. With Docker, the environment inside the container always stays the same. This means that if we can run our service on a laptop, it will work anywhere else.

## 5.4. Deployment

We don’t run production services on our laptops; we need special servers for that.

Some services you can use are AWS, Google Cloud, Microsoft Azure, and Digital Ocean. I won't do this section for now, so let's move on.